In [136]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [137]:
out='../output/'
pref='dam'
inp='../input/monthly/' + pref + '/'


In [143]:
dat=pd.DataFrame(columns=['date','hour','price','volume'])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    print(f.split('_'))
    if f.split('_')[0]==pref:
        if f.split('.')[1]=='csv':
            data = pd.read_csv(join(inp, f))
            dat=pd.concat([dat,data])
            
dat['date']=pd.to_datetime(dat.date,format="%Y-%m-%d")

dat.sort_index(inplace=True)
dat       

['dam', '2016', '01.csv']
['dam', '2016', '01.json']
['dam', '2016', '02.csv']
['dam', '2016', '02.json']
['dam', '2016', '03.csv']
['dam', '2016', '03.json']
['dam', '2016', '04.csv']
['dam', '2016', '04.json']
['dam', '2016', '05.csv']
['dam', '2016', '05.json']
['dam', '2016', '06.csv']
['dam', '2016', '06.json']
['dam', '2016', '07.csv']
['dam', '2016', '07.json']
['dam', '2016', '08.csv']
['dam', '2016', '08.json']
['dam', '2016', '09.csv']
['dam', '2016', '09.json']
['dam', '2016', '10.csv']
['dam', '2016', '10.json']
['dam', '2016', '11.csv']
['dam', '2016', '11.json']
['dam', '2016', '12.csv']
['dam', '2016', '12.json']
['dam', '2017', '01.csv']
['dam', '2017', '01.json']
['dam', '2017', '02.csv']
['dam', '2017', '02.json']
['dam', '2017', '03.csv']
['dam', '2017', '03.json']
['dam', '2017', '04.csv']
['dam', '2017', '04.json']
['dam', '2017', '05.csv']
['dam', '2017', '05.json']
['dam', '2017', '06.csv']
['dam', '2017', '06.json']
['dam', '2017', '07.csv']
['dam', '2017', '07.

C:\Users\Andy Fleming\AppData\Local\Temp\ipykernel_25276\2425000150.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dat=pd.concat([dat,data])


,date,hour,price,volume
0,2022-12-01,0,226.10,27.0
0,2019-08-01,0,54.05,485.9
0,2018-11-01,0,29.98,463.0
0,2019-05-01,0,37.15,220.5
0,2024-05-01,0,25.03,58.0
...,...,...,...,...
1268,2024-01-31,19,199.94,68.0
1269,2024-01-31,20,200.25,80.0
1270,2024-01-31,21,199.07,100.0
1271,2024-01-31,22,158.00,101.0


In [144]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly.sort_index(inplace=True)
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly['hour']=hourly['hour'].astype(int)
hourly

,hour,price,volume,year,month,day,day_of_week
date,,,,,,,
2016-01-01,1,40.08,486.6,2016,1,1,4
2016-01-01,7,48.03,350.0,2016,1,1,4
2016-01-01,8,45.10,323.0,2016,1,1,4
2016-01-01,13,45.10,323.0,2016,1,1,4
2016-01-01,3,40.08,496.6,2016,1,1,4
...,...,...,...,...,...,...,...
2024-06-30,14,109.96,33.0,2024,6,30,6
2024-06-30,9,120.32,102.0,2024,6,30,6
2024-06-30,13,109.95,28.0,2024,6,30,6


In [145]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily.sort_index(inplace=True)
daily


volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2024 6     26   1334.0  271031.120       203.171754     312.80     120.05   
           27   1337.0  282978.960       211.652177     306.54     120.04   
           28   1287.0  277650.800       215.734887     306.46     117.04   
           29   2198.0  363924.130       165.570578     308.35     116.90   
           30   2148.0  284966.260       132.665857     250.10     109.95   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2016 1     1     45.982500      502.9       53.0  328.075000 2016-01-01  
           2     54.114167      486.0       40.0  304.979167 2016-01-02  
           3     54.959167      486.0       40.0  299.945833 2016-01-03  
           4     80.164583      486.0        0.0  185.708333 2016-01-04  
           5     84.503333      436.0        0.0  156.266667 2016-01-05  
...                    ...        ...        ...         ...        ...  
2024 6     26   193.988750       93.0        8.0   55.583333 2024-06-26  
           27   204.397917       93.0       33.0   55.708333 2024-06-27  
           28   203.501250       93.0       33.0   53.625000 2024-06-28  
           29   159.089167      178.0       49.0   91.583333 2024-06-29  
           30   131.244583      151.0       28.0   89.500000 2024-06-30  

[3104 rows x 10 columns]

In [146]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,hour,price,volume,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,1,40.08,486.6,2016,1,1,4,19502.928,off
2016-01-01,7,48.03,350.0,2016,1,1,4,16810.500,morning
2016-01-01,8,45.10,323.0,2016,1,1,4,14567.300,morning
2016-01-01,13,45.10,323.0,2016,1,1,4,14567.300,standard
2016-01-01,3,40.08,496.6,2016,1,1,4,19903.728,off
...,...,...,...,...,...,...,...,...,...
2024-06-30,14,109.96,33.0,2024,6,30,6,3628.680,off
2024-06-30,9,120.32,102.0,2024,6,30,6,12272.640,off
2024-06-30,13,109.95,28.0,2024,6,30,6,3078.600,off


In [147]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2024 6     26   1334.0  271031.120       203.171754     312.80     120.05   
           27   1337.0  282978.960       211.652177     306.54     120.04   
           28   1287.0  277650.800       215.734887     306.46     117.04   
           29   2198.0  363924.130       165.570578     308.35     116.90   
           30   2148.0  284966.260       132.665857     250.10     109.95   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2016 1     1     45.982500      502.9       53.0  328.075000 2016-01-01  ...   
           2     54.114167      486.0       40.0  304.979167 2016-01-02  ...   
           3     54.959167      486.0       40.0  299.945833 2016-01-03  ...   
           4     80.164583      486.0        0.0  185.708333 2016-01-04  ...   
           5     84.503333      436.0        0.0  156.266667 2016-01-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2024 6     26   193.988750       93.0        8.0   55.583333 2024-06-26  ...   
           27   204.397917       93.0       33.0   55.708333 2024-06-27  ...   
           28   203.501250       93.0       33.0   53.625000 2024-06-28  ...   
           29   159.089167      178.0       49.0   91.583333 2024-06-29  ...   
           30   131.244583      151.0       28.0   89.500000 2024-06-30  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   3.0                 2.0      3851.5   
           2                   NaN                 NaN      5969.5   
           3                   NaN                 NaN      7198.7   
           4                   3.0                 2.0      3828.0   
           5                   3.0                 2.0      3230.0   
...                            ...                 ...         ...   
2024 6     26                  3.0                 3.0       455.0   
           27                  3.0                 3.0       426.0   
           28                  3.0                 3.0       350.0   
           29                  NaN                 NaN      1370.0   
           30                  NaN                 NaN      2148.0   

                standard_volume  morning_volume  evening_volume   off_value  \
year month day                                                                
2016 1     1             2920.3           993.0           109.0  152306.418   
           2             1350.0             NaN             NaN  271567.021   
           3                NaN             NaN             NaN  351712.733   
           4              464.0           109.0            56.0  179880.680   
           5              368.4           109.0            43.0  136366.420   
...                         ...             ...             ...         ...   
2024 6     26             501.0           194.0           184.0   54725.840   
           27             525.0           192.0           194.0   53621.920   
           28             534.0           209.0           194.0   44354.820   
           29             828.0             NaN             NaN  180133.510   
           30               NaN             NaN             NaN  284966.260   

                standard_value  mo

In [148]:
daily.to_csv(out + pref + '_daily.csv')

In [149]:
hourly.to_csv(out + pref + '_hourly.csv')

In [150]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,1,40.08,486.6,off
2016-01-01,7,48.03,350.0,morning
2016-01-01,8,45.10,323.0,morning
2016-01-01,13,45.10,323.0,standard
2016-01-01,3,40.08,496.6,off
...,...,...,...,...
2024-06-30,14,109.96,33.0,off
2024-06-30,9,120.32,102.0,off
2024-06-30,13,109.95,28.0,off


In [151]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [152]:
monthly=daily.groupby(['year','month']).mean()
monthly=monthly.drop(columns=['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'], axis = 1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly

volume_mean     value_mean  price_max  price_min  price_mean  \
year month                                                                 
2016 1      3877.909677  216538.697903        NaN        NaN   81.784624   
     2      3915.110345  263339.799483        NaN        NaN   91.385273   
     3      3004.254839  192831.319548        NaN        NaN   88.099516   
     4      1934.450000  119384.364433        NaN        NaN   85.320472   
     5      1714.970968   95755.502839        NaN        NaN   78.638495   
...                 ...            ...        ...        ...         ...   
2024 2      4986.931034  401522.993448        NaN        NaN   98.470690   
     3      2599.645161  296836.615806        NaN        NaN  131.941586   
     4      2959.733333  234563.848333        NaN        NaN   85.931083   
     5      2514.645161  127451.500968        NaN        NaN   61.285833   
     6      2157.466667  325214.858000        NaN        NaN  155.957861   

             power_max  power_min  power_mean                date  \
year month                                                          
2016 1      362.651613  17.606452  161.579570 2016-01-16 00:00:00   
     2      367.837931  23.231034  163.129598 2016-02-15 00:00:00   
     3      285.867742  18.232258  125.177285 2016-03-16 00:00:00   
     4      183.970000   7.236667   80.602083 2016-04-15 12:00:00   
     5      165.619355   4.661290   71.457124 2016-05-16 00:00:00   
...                ...        ...         ...                 ...   
2024 2      323.620690  57.206897  207.788793 2024-02-15 00:00:00   
     3      191.258065  19.741935  108.318548 2024-03-16 00:00:00   
     4      196.633333  35.766667  123.322222 2024-04-15 12:00:00   
     5      193.354839  27.935484  104.776882 2024-05-16 00:00:00   
     6      164.633333  32.200000   89.894444 2024-06-15 12:00:00   

            off_price_mean  ...  evening_price_max  standard_price_min  \
year month                  ...                                          
2016 1           52.483265  ...                NaN                 NaN   
     2           57.262857  ...                NaN                 NaN   
     3           55.448623  ...                NaN                 NaN   
     4           53.100462  ...                NaN                 NaN   
     5           44.707955  ...                NaN                 NaN   
...                    ...  ...                ...                 ...   
2024 2           72.624674  ...                NaN                 NaN   
     3          119.986066  ...                NaN                 NaN   
     4           81.491467  ...                NaN                 NaN   
     5           44.166293  ...                NaN                 NaN   
     6          116.695563  ...                NaN                 NaN   

            off_price_min  morning_price_min  evening_price_min  price_stdev  \
year month                                                                     
2016 1                NaN                NaN                NaN          NaN   
     2                NaN                NaN                NaN          NaN   
     3                NaN                NaN                NaN          NaN   
     4                NaN                NaN                NaN          NaN   
     5                NaN                NaN                NaN          NaN   
...                   ...                ...                ...          ...   
2024 2                NaN                NaN                NaN          NaN   
     3                NaN                NaN                NaN          NaN   
     4                NaN                NaN                NaN          NaN   
     5                NaN                NaN                NaN          NaN   
     6                NaN                NaN                NaN          NaN   

            standard_price_stdev  off_price_stdev  morning_price_stdev  \
year month                                     

In [153]:
monthly.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 102 entries, (np.int32(2016), np.int32(1)) to (np.int32(2024), np.int32(6))
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   volume_mean           102 non-null    float64       
 1   value_mean            102 non-null    float64       
 2   price_max             0 non-null      float64       
 3   price_min             0 non-null      float64       
 4   price_mean            102 non-null    float64       
 5   power_max             102 non-null    float64       
 6   power_min             102 non-null    float64       
 7   power_mean            102 non-null    float64       
 8   date                  102 non-null    datetime64[ns]
 9   off_price_mean        102 non-null    float64       
 10  standard_price_mean   102 non-null    float64       
 11  morning_price_mean    102 non-null    float64       
 12  evening_price_mean    10

In [154]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.drop(columns=['date'],axis=1).groupby(['year','month']).sum()[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
    monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.drop(columns=['date'],axis=1).groupby(['year','month']).sum()[['value','morning_value','evening_value','standard_value','off_value']]
    monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.drop(columns=['date'],axis=1).groupby(['year','month']).sum()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

    monthly['price_weight_mean']=monthly['value'] / monthly['volume']
    monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
    monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
    monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
    monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']

    monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
    monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
    monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
    monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']
monthly

volume_mean     value_mean  price_max  price_min  price_mean  \
year month                                                                 
2016 1      3877.909677  216538.697903        NaN        NaN   81.784624   
     2      3915.110345  263339.799483        NaN        NaN   91.385273   
     3      3004.254839  192831.319548        NaN        NaN   88.099516   
     4      1934.450000  119384.364433        NaN        NaN   85.320472   
     5      1714.970968   95755.502839        NaN        NaN   78.638495   
...                 ...            ...        ...        ...         ...   
2024 2      4986.931034  401522.993448        NaN        NaN   98.470690   
     3      2599.645161  296836.615806        NaN        NaN  131.941586   
     4      2959.733333  234563.848333        NaN        NaN   85.931083   
     5      2514.645161  127451.500968        NaN        NaN   61.285833   
     6      2157.466667  325214.858000        NaN        NaN  155.957861   

             power_max  power_min  power_mean                date  \
year month                                                          
2016 1      362.651613  17.606452  161.579570 2016-01-16 00:00:00   
     2      367.837931  23.231034  163.129598 2016-02-15 00:00:00   
     3      285.867742  18.232258  125.177285 2016-03-16 00:00:00   
     4      183.970000   7.236667   80.602083 2016-04-15 12:00:00   
     5      165.619355   4.661290   71.457124 2016-05-16 00:00:00   
...                ...        ...         ...                 ...   
2024 2      323.620690  57.206897  207.788793 2024-02-15 00:00:00   
     3      191.258065  19.741935  108.318548 2024-03-16 00:00:00   
     4      196.633333  35.766667  123.322222 2024-04-15 12:00:00   
     5      193.354839  27.935484  104.776882 2024-05-16 00:00:00   
     6      164.633333  32.200000   89.894444 2024-06-15 12:00:00   

            off_price_mean  ...  evening_hour_count  price_weight_mean  \
year month                  ...                                          
2016 1           52.483265  ...                42.0          55.839026   
     2           57.262857  ...                42.0          67.262421   
     3           55.448623  ...                46.0          64.186073   
     4           53.100462  ...                42.0          61.714888   
     5           44.707955  ...                44.0          55.835058   
...                    ...  ...                 ...                ...   
2024 2           72.624674  ...                42.0          80.515048   
     3          119.986066  ...                42.0         114.183513   
     4           81.491467  ...                44.0          79.251683   
     5           44.166293  ...                46.0          50.683692   
     6          116.695563  ...                60.0         150.739227   

            off_price_weight_mean  standard_price_weight_mean  \
year month                                                      
2016 1                  50.385088                   81.002858   
     2                  56.542839                  106.547628   
     3                  55.124423                   98.717668   
     4                  56.919792                  104.192200   
     5                  49.682582                  106.019186   
...                           ...                         ...   
2024 2                  62.673911                  105.162668   
     3                 108.834982                  119.579062   
     4                  75.424290                   85.515912   
     5                  41.742497                   56.410526   
     6                 117.945515                  172.790203   

            morning_price_weight_mean  evening_price_weight_mean   off_power  \
year month                                                                     
2016 1                      77.850006                 137.720998  272.092493   
     2                     129.026815                 156.801247  275.132530

In [155]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [156]:
monthly.to_csv(out + pref + '_monthly.csv')

In [157]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [158]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(columns=['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'], axis=1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,date,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,205849.119667,53.109272,151.644050,32.422903,63.351653,268.325090,49.245161,155.316725,2020-01-16 08:00:00,...,3033.688889,6.381323e+06,387.888889,295.444444,71.666667,47.777778,199.173310,120.906980,119.761273,62.810797
1,2,203637.127196,64.244502,178.771098,37.191020,75.047502,241.920392,38.790588,137.347141,2020-02-15 00:00:00,...,2511.588889,5.769719e+06,327.666667,251.222222,60.666667,40.444444,167.739325,113.874659,120.295388,62.169960
2,3,163546.990204,66.130952,158.093477,43.430000,75.776154,187.846595,38.867742,105.431944,2020-03-15 16:00:00,...,2586.166667,5.069957e+06,356.333333,276.000000,67.000000,44.666667,123.922643,92.217768,95.263714,57.801017
3,4,157560.333330,55.514554,146.133778,34.005704,63.466752,200.130000,46.563704,122.698858,2020-04-15 04:00:00,...,3260.966667,4.726810e+06,349.333333,264.555556,63.666667,42.444444,137.382656,110.808715,125.324249,76.953437
4,5,134429.060541,49.417741,148.846953,27.491362,58.548541,199.623656,45.116487,113.412321,2020-05-15 16:00:00,...,2816.922222,4.167301e+06,357.111111,275.222222,67.000000,44.666667,135.825320,93.060688,111.629656,62.825420
5,6,214922.591511,72.131562,175.893481,39.601185,81.012071,245.332222,48.704444,133.848935,2020-06-15 04:00:00,...,5677.388889,6.447678e+06,342.222222,249.111111,64.333333,64.333333,164.145509,106.748371,121.818249,88.580679
6,7,271285.391960,69.057623,178.221976,36.117823,78.947067,291.149194,61.005242,166.571337,2020-01-15 00:00:00,...,6788.062500,8.409847e+06,359.000000,254.500000,65.250000,65.250000,207.296046,133.421725,136.336488,103.391073
7,8,291208.708202,68.023807,177.561210,33.910806,78.920287,320.982661,73.892742,183.512702,2020-02-15 00:00:00,...,7647.100000,9.027470e+06,350.000000,259.000000,67.500000,67.500000,230.505748,148.822682,146.198410,112.831341
8,9,307691.781700,66.638608,166.139333,35.925083,78.143436,349.911250,84.945833,206.551649,2020-03-16 12:00:00,...,4791.337500,9.230753e+06,347.375000,265.750000,64.125000,42.750000,258.650279,164.246061,163.724119,113.933010
9,10,340192.326806,74.929945,176.103710,40.389677,85.065570,358.527016,85.191129,222.177705,2020-04-16 00:00:00,...,4050.250000,1.054596e+07,362.500000,272.125000,65.625000,43.750000,274.962337,186.427287,165.897595,92.382107


In [159]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [160]:
hourly

,hour,price,volume,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,1,40.08,486.6,2016,1,1,4,19502.928,off
2016-01-01,7,48.03,350.0,2016,1,1,4,16810.500,morning
2016-01-01,8,45.10,323.0,2016,1,1,4,14567.300,morning
2016-01-01,13,45.10,323.0,2016,1,1,4,14567.300,standard
2016-01-01,3,40.08,496.6,2016,1,1,4,19903.728,off
...,...,...,...,...,...,...,...,...,...
2024-06-30,14,109.96,33.0,2024,6,30,6,3628.680,off
2024-06-30,9,120.32,102.0,2024,6,30,6,12272.640,off
2024-06-30,13,109.95,28.0,2024,6,30,6,3078.600,off


In [161]:
calmonthly_hours=hourly.reset_index().drop(columns=['date','cat'], axis=1).groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(columns=['year','day','day_of_week'],axis=1)
calmonthly_hours

price      volume         value
month hour                                      
1     0      36.642625  211.924917   7582.106525
      1      35.112658  217.616611   7550.222336
      2      34.378372  220.886711   7549.566422
      3      34.140764  222.852492   7592.468402
      4      34.877043  223.776080   7737.967176
...                ...         ...           ...
12    19    159.632419   72.631855   9404.409960
      20    135.432581   86.777419   8224.346786
      21    106.667056  136.574597  10570.280742
      22     59.507782  202.443145   9090.053879
      23     49.863750  213.235887   8716.215597

[288 rows x 3 columns]

In [162]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [163]:
yearly_calmonthly_hours=hourly.reset_index().drop(columns=['date','cat'], axis=1).groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(columns=['day','day_of_week'], axis=1)
yearly_calmonthly_hours

price      volume         value
year month hour                                      
2016 1     0      48.541613  345.325806  16585.471323
           1      48.442258  348.409677  16691.800032
           2      48.383226  350.206452  16751.135161
           3      48.492903  357.500000  17144.462097
           4      48.556129  358.341935  17206.331742
...                     ...         ...           ...
2024 6     19    249.598000   77.266667  18252.560667
           20    200.116000  101.866667  19494.609667
           21    164.225667   84.833333  13536.304333
           22    119.260667   91.466667  10467.632333
           23    109.977000   91.333333   9721.441000

[2448 rows x 3 columns]

In [164]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [143]:
from statsmodels.tsa.stattools import pacf

In [144]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.0,1.0,1.000000
1,0.575693,NaN,NaN,0.719482
2,-0.141688,NaN,NaN,0.156318
3,0.094764,NaN,NaN,0.076828
4,0.372824,NaN,NaN,0.146447
5,0.214987,NaN,NaN,-0.107136
6,0.050937,NaN,NaN,-0.169003
7,0.017093,NaN,NaN,0.142308
8,0.053606,NaN,NaN,-0.010914
9,-0.030040,NaN,NaN,0.085581


In [145]:
pacfs.to_csv(out + pref + '_pacfs.csv')